In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import copy

import pandas as pd
import seaborn as sns
from glob import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm
#from tensorflow.keras.layers.merge import Maximum, Minimum

#import tensorflow_docs as tfdocs
#import tensorflow_docs.plots
#import tensorflow_docs.modeling

In [2]:
def shift_cdf(a):
    for i in range(a.shape[1]-1,3,-1):
        a[:,i] = a[:,i] - a[:,i-1]
        
    return a

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
data_directory = './training_data'

os.chdir(data_directory)

cdf_training_files = glob('*survey.csv')

In [5]:
total_training_set = {}

c = np.linspace(1,99,99,dtype=int)
column_names = [str(ic) for ic in c]

gs = np.linspace(1,11,11,dtype=int)
g_names = ['g'+str(i) for i in gs]

#column_names = ['c']+g_names+['g']+column_names
column_names = ['c']+['g']+column_names
#print(column_names)

for i in range(0,len(cdf_training_files)):
    if i==0:
        a = pd.read_csv(cdf_training_files[i],header=None).to_numpy()
        
        concs = np.zeros(shape=(a.shape[0],1))
        concs[:,0] = a[:,0]
        
        response_labels = np.zeros(shape=(a.shape[0],a.shape[0]))

        for j in range(0,a.shape[0]):
            response_labels[j,:] = a[:,1]
         
        for j in range(0,a.shape[0]):
            if a[j,2]==0.0:
                a[j,2] = 0.5*a[j,3]
        #print(concs.shape,response_labels.shape,a[:,1:].shape)
            
        #data_array = np.zeros(shape=(a.shape[0],(a.shape[1]+response_labels.shape[1])))
        
        #aa = shift_cdf(a)
        
        #data_array = np.concatenate((concs,aa[:,1:]),axis=1)
        mean_array = np.zeros(shape=(a[:,1].shape[0],1))
        mean_array[:,0] = a[:,1]
        data_array = np.concatenate((concs,mean_array,a[:,3:-1]),axis=1)
        
        #print(data_array)
    else:
        a = pd.read_csv(cdf_training_files[i],header=None).to_numpy()
        
        concs = np.zeros(shape=(a.shape[0],1))
        concs[:,0] = a[:,0]
        
        response_labels = np.zeros(shape=(a.shape[0],a.shape[0]))

        for j in range(0,a.shape[0]):
            response_labels[j,:] = a[:,1]
        
        for j in range(0,a.shape[0]):
            if a[j,2]==0.0:
                a[j,2] = 0.5*a[j,3]
            
        #aa = shift_cdf(a)
            
        #this_array = np.concatenate((concs,aa[:,1:]),axis=1)
        mean_array = np.zeros(shape=(a[:,1].shape[0],1))
        mean_array[:,0] = a[:,1]
        this_array = np.concatenate((concs,mean_array,a[:,3:-1]),axis=1)
        
        #print(i,cdf_training_files[i],this_array.shape,data_array.shape)
        #sys.stdout.flush()
            
        data_array = np.concatenate((data_array,this_array),axis=0)
        
total_training_set = pd.DataFrame(data_array,index=[i for i in range(data_array.shape[0])],columns=column_names)
#total_training_set[i] = pd.read_csv(cdf_training_files[i],names=column_names)
#print(total_training_set['g'])
print(data_array.shape)
#print(list(total_training_set))

(1200, 101)


In [6]:
training_labels = {}
training_sets = {}

keyset = list(np.linspace(2,data_array.shape[1]-1,99,dtype=int))
keyset.reverse()
#print(keyset)

for i in keyset:
    #print(i,column_names[i])
    #training_labels[i-13] = total_training_set.pop(column_names[i])
    #training_sets[i-13] = copy.deepcopy(total_training_set)
    #print(column_names[i])
    training_labels[i-1] = total_training_set.pop(column_names[i])
    training_sets[i-1] = copy.deepcopy(total_training_set)
    
    if i>=3:   
        #if i==3:
        #    print(training_labels[i-2])
            
        training_labels[i-1] = training_labels[i-1] - total_training_set[str(i-2)]
        
        #if i==3:
        #    print(training_labels[i-2])
        #    print(total_training_set[str(i-3)])
keyset.reverse()

keyset = np.array(keyset,dtype=int) - 2*np.ones(shape=(len(keyset),),dtype=int)

**Train and save models**

In [ ]:
data_directory = '../models/'
os.chdir(data_directory)

model_name = '3-layers-64-32-16'

try:
    os.chdir(model_name)
except OSError:
    os.mkdir(model_name)
    os.chdir(model_name)

EPOCHS = 5000

training_loss = np.zeros(shape=(99,3))
validation_loss = np.zeros(shape=(99,3))

for i in range(1,100):
    def build_model(qi):
        if qi>=0:
            model = keras.Sequential([
            layers.Dense(64, activation='relu', input_shape=[len(training_sets[i].keys())]),
            layers.Dense(32, activation='relu'),
            layers.Dense(16, activation='relu'),
            layers.Dense(1)
            ])
        """
        elif qi>=50:
            model = keras.Sequential([
            layers.Dense(32, activation='relu', input_shape=[len(training_sets[i].keys())]),
            layers.Dense(32, activation='relu'),
            #layers.Dense(64, activation='relu'),
            layers.Dense(1)
            ])
        elif qi>=25:
            model = keras.Sequential([
            layers.Dense(64, activation='relu', input_shape=[len(training_sets[i].keys())]),
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(1)
            ])
        else:
            model = keras.Sequential([
            layers.Dense(128, activation='relu', input_shape=[len(training_sets[i].keys())]),
            layers.Dense(128, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(1)
            ])
        """

        optimizer = tf.keras.optimizers.RMSprop(0.002,0.8,centered=True)

        model.compile(loss='mse',optimizer=optimizer,metrics=['mae', 'mse'])
        
        return model
    
    model = build_model(i)

    #model.summary()
    
    j = i - 1
    
    history = model.fit(training_sets[i], training_labels[i],
      epochs=EPOCHS, validation_split = 0.2, verbose=0, batch_size=12, shuffle=True)#callbacks=[tfdocs.modeling.EpochDots()])
    
    loss_array = np.zeros(shape=(len(history.history['loss']),3))
    loss_array[:,0] = np.linspace(1,loss_array.shape[0],loss_array.shape[0])
    loss_array[:,1] = np.array(history.history['loss'])
    loss_array[:,2] = np.array(history.history['val_loss'])

    np.savetxt('loss_array'+str(i)+'.csv',loss_array,delimiter=',')
        
    training_loss[j,0] = history.history['loss'][0]
    training_loss[j,1] = history.history['loss'][-1]
    training_loss[j,2] = history.history['loss'][-1]/history.history['loss'][0]
    
    validation_loss[j,0] = history.history['val_loss'][0]
    validation_loss[j,1] = history.history['val_loss'][-1]
    validation_loss[j,2] = history.history['val_loss'][-1]/history.history['val_loss'][0]
    
    print('Loss(training, validation): ',training_loss[j,2],validation_loss[j,2])
     
    model_name = 'model_'+str(i)+'.h5'
    model.save(model_name)
    
    print(i,' completed.')

Loss(training, validation):  0.6121096228382459 2.335917698230069
1  completed.
Loss(training, validation):  0.7180448476133896 0.6026719647391681
2  completed.
Loss(training, validation):  0.23154507984075584 0.02291020480047681
3  completed.
Loss(training, validation):  0.20446610929887024 0.027736565018240924
4  completed.
Loss(training, validation):  0.09026503043396016 0.006843921662764236
5  completed.
Loss(training, validation):  0.008207445795734408 0.002736386739885441
6  completed.
Loss(training, validation):  0.0023944565950288596 0.09929183702662724
7  completed.
Loss(training, validation):  0.00835044381797208 0.054913659763183006
8  completed.
Loss(training, validation):  0.0013989165986884472 0.1731679218923829
9  completed.
Loss(training, validation):  0.00019787539857491325 0.00026188373516625727
10  completed.
Loss(training, validation):  0.0024761885697409764 0.03619549411250516
11  completed.
Loss(training, validation):  0.00013296766194312125 0.029302021333767934
1

In [ ]:
np.savetxt('validation_loss.csv',validation_loss,delimiter=',')
np.savetxt('training_loss.csv',training_loss,delimiter=',')